In [1]:
cd '/content/drive/My Drive/CEWIT Hack'

/content/drive/My Drive/CEWIT Hack


In [6]:
pip install flask-ngrok

In [7]:
pip install fpdf

  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40720 sha256=aa0afb261410a919730e1aff435cff972bc144131911c88a03f9d24cdd86fe69
  Stored in directory: /root/.cache/pip/wheels/9a/e9/77/4554ff5c99bc3f487c8d69620d8c41d99d54e9c54ab20ef4c9
Successfully built fpdf


In [21]:
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np
from flask import Flask, request
from flask_ngrok import run_with_ngrok

app=Flask(__name__)
run_with_ngrok(app)

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)


@app.route('/process-request', methods=["POST"])
def call_model():
  name = request.args.get('name') #if key doesn't exist, returns None
  gender = request.args['gender'] #if key doesn't exist, returns a 400, bad request error
  age = request.args.get('age')
  pain_intensity = request.args.get('painintensity')
  patient_desc = request.args.get('patientdesc')
  phone_number = request.args.get('phonenumber')
   
  # Load the model
  model = tensorflow.keras.models.load_model('keras_model.h5')

  # Create the array of the right shape to feed into the keras model
  # The 'length' or number of images you can put into the array is
  # determined by the first position in the shape tuple, in this case 1.
  data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

  # Replace this with the path to your image
  image = Image.open('test_photo2.jpg')

  #resize the image to a 224x224 with the same strategy as in TM2:
  #resizing the image to be at least 224x224 and then cropping from the center
  size = (224, 224)
  image = ImageOps.fit(image, size, Image.ANTIALIAS)

  #turn the image into a numpy array
  image_array = np.asarray(image)

  # display the resized image
  image.show()

  # Normalize the image
  normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

  # Load the image into the array
  data[0] = normalized_image_array

  # run the inference
  prediction = model.predict(data)
  print(prediction)
  disease_detected=np.argmax(prediction)

  category_={}
  doctor_={}
  appointment_={}
  addr_="700 Health Sciences Drive, StonyBrook, NY, 11790"

  if disease_detected==0:
    category_="Category Skin Cuts"
    doctor_="Doctor Available : Dr. John Smith"
    appointment_="Appointment : 4:00pm"
  elif disease_detected==1:
    category_="Category Skin Burn"
    doctor_="Doctor Available : Dr. Rax Ben"
    addr_="Appointment : 4:15pm"
  elif disease_detected==2:
    category_="Category Skin Acne"
    doctor_="Doctor Available : Dr. Hyu Shi"
    appointment_="Appointment : 4:30pm"
  elif disease_detected==3:
    category_="Category Skin Blister"
    doctor_="Doctor Available : Dr. Brain Lui"
    appointment_="Appointment : 5:00pm"
  elif disease_detected==4:
    category_="Category Skin Rashes"
    doctor_= "Doctor Available : Dr. Maxwell Shaw"
    appointment_="Appointment : 5:30pm"
  elif disease_detected==5:
    category_="Category Skin Insect Bites"
    doctor_="Doctor Available : Dr. Jameson Thi"
    appointment_="Appointment : 8:00pm"
  else:
    category_="Category Not Found!!!"

  print(category_)
  print(doctor_)
  print(appointment_)
  print(addr_)
  print(name)
  print(gender)
  print(age)
  print(pain_intensity)
  print(patient_desc)
  print(phone_number)
  
  name="Name: " + name
  gender="Gender : " + gender
  age="Age : " + age
  pain_intensity="Pain intensity : " + pain_intensity
  patient_desc="Patient description : " + patient_desc
  phone_number="Phone number : " + phone_number

  if pain_intensity == '':
    pain_intensity='4'
  if patient_desc == '':
    patient_desc='I am having something.'
  if phone_number=='':
    phone_number='9991111234'

  from fpdf import FPDF
  pdf = FPDF()
  pdf.add_page()
  pdf.set_font("Arial", size=12)
  pdf.line(10, 10, 10, 1000)
  pdf.set_line_width(1)
  pdf.set_draw_color(0, 0, 0)
  pdf.line(10, 20, 1000, 20)
  pdf.line(10, 150, 1000, 150)

  pdf.cell(200, 10, txt="Patient's Details!", ln=1, align="C")
  pdf.cell(200, 10, txt=name , ln=1, align="L")
  pdf.cell(200, 10, txt=gender, ln=1, align="L")
  pdf.cell(200, 10, txt=age, ln=1, align="L")
  pdf.cell(200, 10, txt=pain_intensity, ln=1, align="L")
  pdf.cell(200, 10, txt=patient_desc, ln=1, align="L")
  pdf.cell(200, 10, txt=phone_number, ln=1, align="L")
  pdf.ln(80)
  image_path='test_photo2.jpg'
  pdf.image(image_path, x=50, y=80, w=50)
  pdf.cell(200, 10, txt="{}".format(image_path), ln=1)

  pdf.cell(200, 10, txt="Doctor's Details!", ln=1, align="C")
  pdf.cell(200, 10, txt=category_, ln=1, align="L")
  pdf.cell(200, 10, txt=doctor_, ln=1, align="L")
  pdf.cell(200, 10, txt=appointment_, ln=1, align="L")
  pdf.cell(200, 10, txt=addr_, ln=1, align="L")
  pdf.output("simple_demo_4.pdf")
  print(name,age,gender)

  return('Success')


#if __name__ == '__main__':
app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://96ab4d2e.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [16/Feb/2020 08:58:06] "GET / HTTP/1.1" 404 -


127.0.0.1 - - [16/Feb/2020 09:02:33] "POST /process-request?name=John%20Oliver&gender=female&age=56&phonenumber=98546525&painintensity=4&patientdesc=I%20am%20having%20rashes HTTP/1.1" 200 -


[[0.0000049  0.00040313 0.00000663 0.99917126 0.00000009 0.00041399]]
Category Skin Blister
Doctor Available : Dr. Brain Lui
Appointment : 5:00pm
700 Health Sciences Drive, StonyBrook, NY, 11790
John Oliver
female
56
4
I am having rashes
98546525
Name: John Oliver Age : 56 Gender : female


127.0.0.1 - - [16/Feb/2020 09:14:24] "POST /process-request?name=Ruby&gender=female&age=19&phonenumber=98564532&painintensity=4&patientdesc=I%20am%20having%20rashes HTTP/1.1" 200 -


[[0.0000049  0.00040313 0.00000663 0.99917126 0.00000009 0.00041399]]
Category Skin Blister
Doctor Available : Dr. Brain Lui
Appointment : 5:00pm
700 Health Sciences Drive, StonyBrook, NY, 11790
Ruby
female
19
4
I am having rashes
98564532
Name: Ruby Age : 19 Gender : female


In [0]:
@app.route('/process-request')
def call_model():
   name = request.args.get('name') #if key doesn't exist, returns None
   gender = request.args['gender'] #if key doesn't exist, returns a 400, bad request error
   age = request.args.get('age')
   pain_intensity = request.args.get('pain-intensity')
   patient_desc = request.args.get('patient_desc')
   